In [79]:
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
import statsmodels.formula.api as smf

In [80]:
df_train = pd.read_excel('gl_olympic_tokyo_2020_hackathon_final_feb-b_g9_sat_mandeep_3pm.xlsx', sheet_name = 'train_data')
df_train.head()

,country_id,country_name,medals_won,total_medals,medals_per_edition,prev_medals_1,prev_medals_2,atheletes,health,gdp_pc
0,20210001,Afghanistan,0,2,0.1538,1,1,3,60.188671,509.22
1,20210002,Albania,0,0,0.0000,0,0,6,202.013321,4124.06
2,20210003,Algeria,2,15,1.2500,1,2,64,260.772552,3946.45
3,20210004,American Samoa,0,0,0.0000,0,0,4,0.000000,12038.25
4,20210005,Andorra,0,0,0.0000,0,0,4,2361.217529,37475.64


In [81]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country_id          615 non-null    int64  
 1   country_name        615 non-null    object 
 2   medals_won          615 non-null    int64  
 3   total_medals        615 non-null    int64  
 4   medals_per_edition  615 non-null    float64
 5   prev_medals_1       615 non-null    int64  
 6   prev_medals_2       615 non-null    int64  
 7   atheletes           615 non-null    int64  
 8   health              615 non-null    float64
 9   gdp_pc              615 non-null    float64
dtypes: float64(3), int64(6), object(1)
memory usage: 48.2+ KB


In [82]:
df_train.columns

Index(['country_id', 'country_name', 'medals_won', 'total_medals',
       'medals_per_edition', 'prev_medals_1', 'prev_medals_2', 'atheletes',
       'health', 'gdp_pc'],
      dtype='object')

In [98]:
f = 'medals_won~ total_medals+medals_per_edition+prev_medals_1+prev_medals_2+atheletes+health+gdp_pc'

lrm = smf.ols(formula = f, data = df_train).fit()

pd.DataFrame(lrm.params).to_excel('coefficients.xlsx', sheet_name = 'coefficients')

lrm.params

Intercept            -0.240597
total_medals          0.007464
medals_per_edition   -0.088797
prev_medals_1         0.776450
prev_medals_2         0.001168
atheletes             0.024024
health               -0.000069
gdp_pc               -0.000004
dtype: float64

In [99]:
print(lrm.summary())

                            OLS Regression Results                            
Dep. Variable:             medals_won   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     1378.
Date:                Fri, 23 Jul 2021   Prob (F-statistic):               0.00
Time:                        23:26:59   Log-Likelihood:                -1590.5
No. Observations:                 615   AIC:                             3197.
Df Residuals:                     607   BIC:                             3232.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2406      0

In [100]:
df_test = pd.read_excel('gl_olympic_tokyo_2020_hackathon_final_feb-b_g9_sat_mandeep_3pm.xlsx', sheet_name = 'test_data')
df_test.head()

,country_id,country_name,total_medals,medals_per_edition,prev_medals_1,prev_medals_2,atheletes,health,gdp_pc
0,20210001,Afghanistan,2,0.1429,0,1,5,49.842609,508.81
1,20210002,Albania,0,0.0000,0,0,9,274.914093,5215.28
2,20210003,Algeria,17,1.3077,2,1,44,255.869431,3310.39
3,20210004,American Samoa,0,0.0000,0,0,6,0.000000,11413.66
4,20210005,Andorra,0,0.0000,0,0,2,2821.801270,41804.18


In [101]:
df_test = df_test.drop(['country_id', 'country_name'], axis = 1)

In [106]:
pred_medals = pd.DataFrame()
pred_medals['medals_tally'] = round(lrm.predict(df_test),0)
pred_medals.head()

,medals_tally
0,-0.0
1,-0.0
2,2.0
3,-0.0
4,-1.0


In [104]:
pred_medals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   medals_tally  205 non-null    float64
dtypes: float64(1)
memory usage: 3.2 KB


In [105]:
pred_medals = pred_medals.replace(-0.0, 0)
pred_medals['medals_tally'] = np.where(pred_medals['medals_tally'] < 0, 0, pred_medals['medals_tally'])
pred_medals.to_excel('predicted_medals_tally_tokyo_olympics_2020.xlsx', sheet_name = 'pred_medals_tally')
pred_medals.head()

,medals_tally
0,0.0
1,0.0
2,2.0
3,0.0
4,0.0


In [97]:
pred_medals['medals_tally'].sum()

1010.0